````bash
#!/bin/bash
# Genera un nombre de bucket único
BUCKET_NAME="usersjsonapp-$(date +'%Y%m%d%H%M%S')"

echo "Nombre del bucket será: $BUCKET_NAME"

# Una vez que tienes un nombre único disponible, crea el bucket
aws s3api create-bucket --bucket "$BUCKET_NAME" --create-bucket-configuration LocationConstraint=eu-west-3

echo "bucket creado: $BUCKET_NAME"

# Crear una tabla en DynamoDB
aws dynamodb create-table \
  --table-name usersTable \
  --attribute-definitions AttributeName=ID,AttributeType=N \
  --key-schema AttributeName=ID,KeyType=HASH \
  --provisioned-throughput ReadCapacityUnits=5,WriteCapacityUnits=5 \
  --region eu-west-3

# Esperar a que la tabla de DynamoDB esté en estado "ACTIVE"
while true; do
  TABLE_STATUS=$(aws dynamodb describe-table --table-name usersTable --query 'Table.TableStatus' --output text)
  if [ "$TABLE_STATUS" = "ACTIVE" ]; then
    break
  else
    echo "Esperando a que la tabla usersTable esté en estado 'ACTIVE'..."
    sleep 5  # Espera 5 segundos antes de volver a verificar
  fi
done

# Creamos un rol para Lambda
aws iam create-role \
  --role-name RolLambdaCLI \
  --assume-role-policy-document '{
    "Version": "2012-10-17",
    "Statement": [
      {
        "Effect": "Allow",
        "Principal": {
          "Service": "lambda.amazonaws.com"
        },
        "Action": "sts:AssumeRole"
      }
    ]
  }'


# Asignamos políticas al rol
aws iam attach-role-policy --role-name RolLambdaCLI --policy-arn arn:aws:iam::aws:policy/AmazonS3FullAccess
aws iam attach-role-policy --role-name RolLambdaCLI --policy-arn arn:aws:iam::aws:policy/AmazonDynamoDBFullAccess

IAM_ROLE_ARN=$(aws iam get-role --role-name RolLambdaCLI --query 'Role.Arn' --output text)

# Crear el archivo Python para la función Lambda
cat > mi-funcion.py <<EOF
import json
import boto3

# Creando instancias de tabla DynamoDB y S3
s3 = boto3.client('s3', region_name="eu-west-3")
dynamodb = boto3.resource('dynamodb', region_name='eu-west-3')

# Nombre de la tabla en DynamoDB
table = dynamodb.Table('usersTable')

def lambda_handler(event, context):
    # Obtiene el nombre del bucket y la clave del archivo JSON del evento de S3
    bucket = event['Records'][0]['s3']['bucket']['name']
    key = event['Records'][0]['s3']['object']['key']

    # Descarga el archivo JSON desde S3
    response = s3.get_object(Bucket=bucket, Key=key)
    json_data = response['Body'].read().decode('utf-8')

    # Parsea el JSON
    data = json.loads(json_data)

    # Inserta los datos en DynamoDB
    response = table.put_item(
        Item={
            'ID': data['ID'],
            'Nombre': data['Nombre'],
            'Correo electrónico': data['Correo electrónico'],
            'Fecha de registro': data['Fecha de registro']
        }
    )
    return {
        'statusCode': 200,
        'body': json.dumps('Datos guardados en DynamoDB exitosamente.')
    }
EOF

# Instalamos la utilidad "zip"
sudo apt-get install zip -y

# Empaquetar el archivo Python en un archivo ZIP
zip mi-archivo-zip.zip mi-funcion.py

# Crear la función Lambda
aws lambda create-function \
  --function-name reads3_writedynamo \
  --runtime python3.8 \
  --role $IAM_ROLE_ARN \
  --handler mi-funcion.lambda_handler \
  --zip-file fileb://mi-archivo-zip.zip

LAMBDA_FUNCTION_ARN=$(aws lambda get-function-configuration --function-name reads3_writedynamo --query 'FunctionArn' --output text)

aws s3api put-bucket-notification-configuration --bucket $BUCKET_NAME --notification-configuration "{
   \"LambdaFunctionConfigurations\": [{
     \"LambdaFunctionArn\": \"$LAMBDA_FUNCTION_ARN\",
     \"Events\": [\"s3:ObjectCreated:*\"]
   }]
}"



# Crear un rol para EC2 y asignar políticas
aws iam create-role \
  --role-name EC2DynamoS3Role \
  --assume-role-policy-document '{
    "Version": "2012-10-17",
    "Statement": [
      {
        "Effect": "Allow",
        "Principal": {
          "Service": "ec2.amazonaws.com"
        },
        "Action": "sts:AssumeRole"
      }
    ]
  }'


# Asignar políticas al rol de EC2
aws iam attach-role-policy --role-name EC2DynamoS3Role --policy-arn arn:aws:iam::aws:policy/AmazonS3FullAccess
aws iam attach-role-policy --role-name EC2DynamoS3Role --policy-arn arn:aws:iam::aws:policy/AmazonDynamoDBFullAccess

# Obtener el ARN del rol de EC2
EC2_ROLE_ARN=$(aws iam get-role --role-name EC2DynamoS3Role --query 'Role.Arn' --output text)

# Obtener el AMI de UBUNTU
AMI_ID=$(aws ec2 describe-images --output json --region eu-west-3 --filters "Name=name,Values=ubuntu/images/hvm-ssd/ubuntu-jammy-22.04-amd64-server*" --query 'Images | sort_by(@, &CreationDate) | [-1].ImageId' --output text)

# Crear la instancia EC2 con el rol asignado
aws ec2 run-instances \
  --image-id "$AMI_ID" \
  --instance-type t2.micro \
  --iam-instance-profile Arn="$EC2_ROLE_ARN" \
  --region eu-west-3

# Nombre de la clave .pem
KEY_NAME="ssh-key"

# Crear una nueva clave .pem si no existe
if [ ! -f "$KEY_NAME.pem" ]; then
  aws ec2 create-key-pair --key-name $KEY_NAME --query 'KeyMaterial' --output text > $KEY_NAME.pem
  chmod 400 $KEY_NAME.pem
fi

# Obtener la dirección IP pública de la instancia
PUBLIC_IP=$(aws ec2 describe-instances --instance-ids $INSTANCE_ID --query 'Reservations[0].Instances[0].PublicIpAddress' --output text)

# Conectar a la instancia
ssh -i $KEY_NAME.pem ubuntu@$PUBLIC_IP

# Crear el archivo Python users-app.py en la instancia
cat > users-app.py <<EOF
import json
import boto3
import dash
from dash import dcc
from dash import html
from dash import dash_table
import random
import datetime

# Creamos una aplicación Dash
app = dash.Dash(__name__, suppress_callback_exceptions=True)

# Creamos un cliente de boto3 para acceder a S3
s3 = boto3.client('s3', region_name="eu-west-3")
today = datetime.date.today().strftime('%Y-%m-d')

# Configuramos la conexión a la tabla de DynamoDB
dynamodb = boto3.resource('dynamodb', region_name='eu-west-3')
tabla_usuarios = dynamodb.Table('usersTable')

# Función para obtener los datos de la tabla de DynamoDB
def obtener_datos_dynamodb():
    response = tabla_usuarios.scan()
    items = response['Items']
    return items

# Definimos el diseño general de la aplicación
app.layout = html.Div([
    html.H1('Menú de Navegación'),  # Título de la página

    # Menú de navegación
    dcc.Link('Formulario de Usuarios', href='/formulario'),  # Enlace al formulario
    html.Br(),  # Salto de línea
    dcc.Link('Tabla de Usuarios', href='/tabla_usuarios'),  # Enlace a la tabla de usuarios
    html.Br(),  # Salto de línea

    # Aquí se mostrará el contenido de las páginas
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])

# Callback para cargar el contenido de las páginas
@app.callback(
    dash.dependencies.Output('page-content', 'children'),
    [dash.Input('url', 'pathname')]
)
def display_page(pathname):
    if pathname == '/formulario':
        # Si el usuario navega al formulario, muestra el contenido del formulario
        return html.Div([
            html.H1('Formulario de Usuarios'),
            dcc.Input(id='nombre', type='text', placeholder='Nombre', value=''),
            dcc.Input(id='email', type='email', placeholder='Email', value=''),
            html.Button('Enviar', id='submit-button', n_clicks=0),
            html.Div(id='output-container-button', children='Hit the button to update.')
        ])
    elif pathname == '/tabla_usuarios':
        # Si el usuario navega a la tabla de usuarios, muestra el contenido de la tabla
        data = obtener_datos_dynamodb()
        return html.Div([
            html.H1('Tabla de Usuarios'),
            dash_table.DataTable(
                columns=[{'name': key, 'id': key} for key in data[0].keys()],
                data=data
            )
        ])

# Ruta para manejar la subida de datos del formulario
@app.callback(
    dash.dependencies.Output('output-container-button', 'children'),
    [dash.Input('submit-button', 'n_clicks'),
    dash.State('nombre', 'value'),
    dash.State('email', 'value')]
)
def submit_form(n_clicks, nombre, email):
    if n_clicks > 0:  # Verifica si se hizo clic en el botón "Enviar"
        # Obtenemos los datos del formulario
        usuario = {
            'ID': random.randint(100000, 999999),
            'Nombre': nombre,
            'Correo electrónico': email,
            'Fecha de registro': today
        }
        # Guarda los datos en un archivo JSON en S3
        s3.put_object(Bucket='$BUCKET_NAME', Key=f'usuarios{today}.json', Body=json.dumps(usuario))
        return f'Datos guardados en S3: {usuario}'  # Puede ser útil proporcionar una confirmación
    else:
        return 'Aún no se ha hecho clic en el botón "Enviar"'

if __name__ == '__main__':
    app.run_server(host='0.0.0.0', port=8080, debug=True)
EOF

# Copiar la aplicación users-app.py a la instancia
scp -o "StrictHostKeyChecking no" -i $KEY_NAME.pem users-app.py ubuntu@$PUBLIC_IP:~/users-app.py

# Instalar Python y bibliotecas necesarias en la instancia
ssh -o "StrictHostKeyChecking no" -i $KEY_NAME.pem ubuntu@$PUBLIC_IP "sudo apt-get update && sudo apt-get install -y python3-pip"
ssh -o "StrictHostKeyChecking no" -i $KEY_NAME.pem ubuntu@$PUBLIC_IP "pip3 install boto3 dash"

# Ejecutar la aplicación users-app.py en la instancia
ssh -o "StrictHostKeyChecking no" -i $KEY_NAME.pem ubuntu@$PUBLIC_IP "python3 users-app.py"

````